# 2. Functions

## Item 14: Prefer Exceptions to Returning `None`
### Things to Remember
- Functions that return `None` to indicate special meaning are error prone because `None` and other values (e.g., zero, the empty string) all evaluate to False in conditional expressions.
- Raise exceptions to indicate special situations instead of returning `None`. Expect the caling code to handle exceptions properly when they're documented.

In [1]:
def divide(a, b):
    try:
        return a / b
    except ZeroDivisionError:
        return None

In [6]:
x, y = 5, 0
result = divide(x, y)
if result is None:
    print('Invalid inputs')

Invalid inputs


In [5]:
x, y = 0, 5
result = divide(x, y)
if not result:
    print('Invalid inputs') # This is wrong!

Invalid inputs


Returning `None` from a function is error prone.   
There are two ways to reduce the chance of such errors.

## First way
To split the return value into a two-tuple. 


In [7]:
def divide(a, b):
    try:
        return True, a / b
    except ZeroDivisionError:
        return False, None

In [15]:
x, y = 0, 5 # True, 0.0
#x, y = 5, 0 # False, None and "Invalid inputs"
success, result = divide(x, y)
print(success, result)
if not success:
    print('Invalid inputs')

True 0.0


bad case.

In [16]:
_, result = divide(x, y)
if not result:
    print('Invalid inputs')

Invalid inputs


## Second way (better way)
Never return `None` at all  
Instead, raise an exception up to the caller and make them deal with it.

In [18]:
def divide(a, b):
    '''
    The behavior below should be documented.
    If  the function didn't raise an exception, then the return value must be good.
    '''
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs') from e

In [27]:
x, y = 5, 0 # Invalid inputs
# x, y = 5, 2 # 2.5
try:
    reslut = divide(x, y)
except ValueError as e:
    print(e)
    #print('Invalid inputs')
else:
    print('Results is %.1f' % reslut)
        
        

Invalid inputs


# Item 15: Know How Closures Interact with Variable Scope
## Things to Remember
- Closure functions can refer to variables from any of the scopes in which they were defined.
- By default, closures can't affect enclosing scopes by assigning variables.
- In Python 3, use the `nonlocal` statement to indicate when a closure can modify a variable in its enclosing scopes.
- In Python 2, use a mutable value (like a single-item list) to work around the lack of the `nonlocal` statement.
- Avoid using `nonlocal` statements for anything beyond simple functions.

In [33]:
def sort_priority(values, group):
    def helper(x):
        if x in group:
            return (0, x)
        return (1, x)
    values.sort(key=helper)

In [34]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
sort_priority(numbers, group)
print(numbers)

[2, 3, 5, 7, 1, 4, 6, 8]


- pythonはclosureをサポートしているため、 `hepler` 関数は`sort_priority`の引数である`group`にアクセスすることができる
- 関数は`first-class objects`。
    - 直接関数を参照すること
    - 変数としてアサインすること
    - 他の関数の引数として渡すこと
    - expressionとif分内で比較すること
- pythonが持つtupleの比較ルール


In [36]:
def sort_priority2(numbers, group):
    found = False
    def helper(x):
        if x in group:
            found = True # Seems simple
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

In [37]:
found = sort_priority2(numbers, group)
print('Found:', found)
print(numbers)

Found: False
[2, 3, 5, 7, 1, 4, 6, 8]


The sorted resluts are correct, but the `found` result is wrong.
This is because of `scoping bug`
```python
def sort_priority2(numbers, group):
    found = False          # Scope: 'sort_priority2'
    def helper(x):
        if x in group:
            found = True   # Scope: 'helper' -- Bad!
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found
```

In python3, there is special syntax for getting data out of a closure. The `nonlocal`.

In [38]:
def sort_priority3(numbers, group):
    found = False
    def helper(x):
        nonlocal found
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

In [40]:
found = sort_priority3(numbers, group)
print('Found:', found)
print(numbers)

Found: True
[2, 3, 5, 7, 1, 4, 6, 8]


In [43]:
# avoid using nonlocal and make a class instead.
# this would be longer but much easier to read.
class Sorter(object):
    def __init__(self, group):
        self.group = group
        self.found = False
        
    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0, x)
        return (1, x)
    
sorter = Sorter(group)
numbers.sort(key=sorter)
assert sorter.found is True
print(numbers)

[2, 3, 5, 7, 1, 4, 6, 8]


# Item 16: Consider Generators Instead of Returning Lists
## Things to Remember
- Using generators can be clearer than the alternative of returning lists of accumulated results.
- The iterator returned by a generator produces the set of values passed to `yield` expressions within the generator functions's body.
- Generators can produce a sequence of outputs for arbitrarily large inputs because their working memory doesn't include all inputs and outputs.

In [ ]:
def 